# Flight Status - Final Project

In [1]:
import os
import shutil
import warnings

import pandas as pd
#import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import opendatasets as od

from sqlalchemy import create_engine
from dotenv import load_dotenv


warnings.filterwarnings('ignore')

## TAS-12 Data Acquisition

### Data Acquisition - Use Kaggle API

[How to use Kaggle API - Step-by-step guide](https://www.geeksforgeeks.org/how-to-download-kaggle-datasets-into-jupyter-notebook/)

Since almost all combined CSV files are larger than 60k records and more than 20 columns the best course of action would be to only use one year data, in this case we are going to use the `Combined_Flights_2022.csv` with the shape `(4078318, 61)` and the `Airlines.csv` for labeling if needed

The **Data Acquisition - Use Kaggle API** section can be re-run only in case we need to access the data from the kaggle API again. After this section we cleaned the data and saved it as CSV in the `/data/processed/` folder, accessed from there, then we'll create a DB with 2 tables `airlines` and `flights`, with the corresponding data and we'll work the data by retrieving it from the DB tables

In [2]:
# ONLY RUN ONCE
od.download('https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading flight-delay-dataset-20182022.zip to .\flight-delay-dataset-20182022


100%|██████████| 3.73G/3.73G [01:46<00:00, 37.7MB/s]


In [3]:
# ONLY RUN ONCE
def move_folder():
    source = './flight-delay-dataset-20182022'
    destination = '../data/raw/flight-delay-dataset-20182022'

    shutil.move(source, destination)

In [4]:
# ONLY RUN ONCE
def remove_unnecesary():
    destination = '../data/raw/flight-delay-dataset-20182022'
    try:
        shutil.rmtree(os.path.join(destination, 'raw'))
    except:
        print("Continue")
    finally:
        files = [x for x in os.listdir(destination) if not ('2022.csv' in x or 'Airlines' in x)]
        for f in files:
            os.remove(os.path.join(destination, f))


In [5]:
# ONLY RUN ONCE
move_folder()

In [6]:
# ONLY RUN ONCE
remove_unnecesary()

In [7]:
# ONLY RUN ONCE
for file in os.listdir('../data/raw/flight-delay-dataset-20182022'):
    shutil.move(os.path.join('../data/raw/flight-delay-dataset-20182022/', file), os.path.join('../data/processed'), file)

### Data Acquisition - Clean and Store Data to CSV

In [8]:
data = pd.read_csv('../data/processed/Combined_Flights_2022.csv')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4078318 entries, 0 to 4078317
Data columns (total 61 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   FlightDate                               object 
 1   Airline                                  object 
 2   Origin                                   object 
 3   Dest                                     object 
 4   Cancelled                                bool   
 5   Diverted                                 bool   
 6   CRSDepTime                               int64  
 7   DepTime                                  float64
 8   DepDelayMinutes                          float64
 9   DepDelay                                 float64
 10  ArrTime                                  float64
 11  ArrDelayMinutes                          float64
 12  AirTime                                  float64
 13  CRSElapsedTime                           float64
 14  ActualElapsedTime 

In [10]:
data.describe()

,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,...,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,DistanceGroup,DivAirportLandings
count,4.078318e+06,3.957885e+06,3.957823e+06,3.957823e+06,3.954079e+06,3.944916e+06,3.944916e+06,4.078318e+06,3.944916e+06,4.078318e+06,...,3.955652e+06,3.955652e+06,3.954076e+06,3.954076e+06,4.078318e+06,3.944916e+06,3.944916e+06,3.944916e+06,4.078318e+06,4.078318e+06
mean,1.329587e+03,1.334374e+03,1.601494e+01,1.309049e+01,1.457886e+03,1.578307e+01,1.110075e+02,1.413211e+02,1.358624e+02,7.978657e+02,...,1.697375e+01,1.356576e+03,1.455073e+03,7.894387e+00,1.486058e+03,7.528486e+00,2.164715e-01,-6.256103e-02,3.663516e+00,3.685098e-03
std,4.904801e+02,5.056219e+02,5.231498e+01,5.332016e+01,5.431841e+02,5.198424e+01,6.996246e+01,7.179635e+01,7.185501e+01,5.914742e+02,...,9.495407e+00,5.075580e+02,5.378428e+02,6.663118e+00,5.185078e+02,5.524625e+01,4.118393e-01,2.487442e+00,2.320848e+00,1.141331e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,-7.800000e+01,1.000000e+00,0.000000e+00,8.000000e+00,-4.800000e+01,1.400000e+01,3.100000e+01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.000000e+02,0.000000e+00,-2.000000e+00,1.000000e+00,0.000000e+00
25%,9.140000e+02,9.170000e+02,0.000000e+00,-5.000000e+00,1.046000e+03,0.000000e+00,6.000000e+01,8.900000e+01,8.300000e+01,3.680000e+02,...,1.100000e+01,9.320000e+02,1.044000e+03,4.000000e+00,1.103000e+03,-1.400000e+01,0.000000e+00,-1.000000e+00,2.000000e+00,0.000000e+00
50%,1.320000e+03,1.325000e+03,0.000000e+00,-2.000000e+00,1.500000e+03,0.000000e+00,9.400000e+01,1.240000e+02,1.190000e+02,6.430000e+02,...,1.500000e+01,1.338000e+03,1.456000e+03,6.000000e+00,1.513000e+03,-5.000000e+00,0.000000e+00,-1.000000e+00,3.000000e+00,0.000000e+00
75%,1.735000e+03,1.744000e+03,1.100000e+01,1.100000e+01,1.914000e+03,1.000000e+01,1.410000e+02,1.710000e+02,1.670000e+02,1.035000e+03,...,1.900000e+01,1.758000e+03,1.909000e+03,9.000000e+00,1.920000e+03,1.000000e+01,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00
max,2.359000e+03,2.400000e+03,7.223000e+03,7.223000e+03,2.400000e+03,7.232000e+03,7.270000e+02,6.900000e+02,7.640000e+02,5.095000e+03,...,2.210000e+02,2.400000e+03,2.400000e+03,2.900000e+02,2.359000e+03,7.232000e+03,1.000000e+00,1.200000e+01,1.100000e+01,9.000000e+00


In [11]:
def check_nulls(df):
    # Assuming df is your DataFrame
    # Check for null values in each column
    percentage = 10
    percent = (percentage * len(df)) / 100
    null_counts = df.isnull().sum()

    # Filter columns with null values and print their sum
    columns_with_nulls_ten = null_counts[null_counts > percent]
    columns_with_nulls = null_counts[null_counts > 0]
    if len(columns_with_nulls_ten) > 0:
        for column, count in columns_with_nulls.items():
            print(f"Column '{column}' has {count} null values.")
    else:
        print("The null values in the dataframe don't exceed {percent} values or {percentage}% of the total data".format(percent=percent, percentage=percentage))
        print("Depending on Duplicated values we might want to consider dropping them since that low percentage of null values would hardly make any difference in the EDA or the model creation and prediction")

In [12]:
check_nulls(data)

The null values in the dataframe don't exceed 407831.8 values or 10% of the total data
Depending on Duplicated values we might want to consider dropping them since that low percentage of null values would hardly make any difference in the EDA or the model creation and prediction


In [13]:
# data.duplicated().sum()

In [14]:
def treat_nulls(value):
    if pd.isnull(value):
        return 0
    else:
        return int(value)
    
# Define a function to transform values
def transform_time(value):
    if isinstance(value, int):
        value = str(value)  # Convert integer to string
    value = value.zfill(4)  # Pad with leading zeros if necessary
    if len(value) == 4:
        if int(value) == 2400:
            return '00:00'
        if int(value) < 10:  # For values less than 10
            return f'00:0{value[0]}'
        elif int(value) < 100:  # For values between 10 and 100
            return f'00:{value[:2]}'
        else:  # For values over 100
            return f'{value[:2]}:{value[2:]}'
    else:
        return value[:2] + ':' + value[2:]  # Format as 'HH:MM'

In [15]:
def format_dates():
    data['FlightDate'] = pd.to_datetime(data['FlightDate'])

    # Format time WheelsOff
    data['WheelsOff'] = data['WheelsOff'].apply(treat_nulls)
    data['WheelsOff'] = data['WheelsOff'].apply(transform_time)
    data['WheelsOff'] = pd.to_datetime(data['WheelsOff'], format='%H:%M').dt.time

    # Format time WheelsOn
    data['WheelsOn'] = data['WheelsOn'].apply(treat_nulls)
    data['WheelsOn'] = data['WheelsOn'].apply(transform_time)
    data['WheelsOn'] = pd.to_datetime(data['WheelsOn'], format='%H:%M').dt.time

    # Format time ArrTime
    data['ArrTime'] = data['ArrTime'].apply(treat_nulls)
    data['ArrTime'] = data['ArrTime'].apply(transform_time)
    data['ArrTime'] = pd.to_datetime(data['ArrTime'], format='%H:%M').dt.time


    # Format time DepTime
    data['DepTime'] = data['DepTime'].apply(treat_nulls)
    data['DepTime'] = data['DepTime'].apply(transform_time)
    data['DepTime'] = pd.to_datetime(data['DepTime'], format='%H:%M').dt.time

In [16]:
format_dates()

In [17]:
origins = data[['OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac']]
origins.drop_duplicates(inplace=True)
origins.to_csv('../data/processed/origins.csv', index = False)

In [18]:
destinations = data[['DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac']]
destinations.drop_duplicates(inplace=True)
destinations.to_csv('../data/processed/destinations.csv', index = False)

In [19]:
data.drop(
    columns=[
        'CRSDepTime',
        'ActualElapsedTime',
        'CRSArrTime',
        'OriginAirportSeqID',
        'OriginCityMarketID',
        'Origin',
        'OriginCityName',
        'OriginState',
        'OriginStateFips',
        'OriginStateName',
        'OriginWac',
        'TaxiIn',
        'TaxiOut',
        'ArrDel15',
        'ArrivalDelayGroups',
        'ArrTimeBlk',
        'DistanceGroup',
        'CRSDepTime',
        'DepDel15',
        'DepartureDelayGroups',
        'DepTimeBlk',
        'DestAirportSeqID',
        'DestCityMarketID',
        'Dest',
        'DestCityName',
        'DestState',
        'DestStateFips',
        'DestStateName',
        'DestWac',
        'CRSArrTime',
        'CRSElapsedTime'
    ], 
    inplace=True
)

In [20]:
colms = data.columns

print(colms)

Index(['FlightDate', 'Airline', 'Cancelled', 'Diverted', 'DepTime',
       'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'DestAirportID', 'WheelsOff', 'WheelsOn', 'ArrDelay',
       'DivAirportLandings'],
      dtype='object')


In [21]:
data.to_csv('../data/processed/Combined_Flights_2022.csv', index = False)

In [22]:
data = pd.read_csv('../data/processed/Combined_Flights_2022.csv')
origins = pd.read_csv('../data/processed/origins.csv')
destinations = pd.read_csv('../data/processed/destinations.csv')

In [23]:
data.columns

Index(['FlightDate', 'Airline', 'Cancelled', 'Diverted', 'DepTime',
       'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'DestAirportID', 'WheelsOff', 'WheelsOn', 'ArrDelay',
       'DivAirportLandings'],
      dtype='object')

In [24]:
data.head()

,FlightDate,Airline,Cancelled,Diverted,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,...,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,DestAirportID,WheelsOff,WheelsOn,ArrDelay,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",False,False,11:23:00,0.0,-10.0,12:28:00,0.0,40.0,...,20445,C5,N21144,4301,11921,11292,11:40:00,12:20:00,-17.0,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",False,False,07:28:00,0.0,-4.0,08:48:00,0.0,55.0,...,20445,C5,N16170,4299,12206,12266,07:44:00,08:39:00,-1.0,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",False,False,15:14:00,0.0,-15.0,16:36:00,0.0,47.0,...,20445,C5,N21144,4298,11413,11292,15:35:00,16:22:00,-3.0,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",False,False,14:30:00,0.0,-5.0,15:47:00,0.0,57.0,...,20445,C5,N11184,4296,12266,11973,14:46:00,15:43:00,-18.0,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",False,False,11:35:00,0.0,0.0,12:51:00,6.0,49.0,...,20445,C5,N17146,4295,11413,11292,11:54:00,12:43:00,6.0,0


In [25]:
origins.head()

,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac
0,11921,1192102,31921,GJT,"Grand Junction, CO",CO,8,Colorado,82
1,12206,1220605,32206,HRL,"Harlingen/San Benito, TX",TX,48,Texas,74
2,11413,1141307,30285,DRO,"Durango, CO",CO,8,Colorado,82
3,12266,1226603,31453,IAH,"Houston, TX",TX,48,Texas,74
4,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82


In [26]:
destinations.head()

,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac
0,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82
1,12266,1226603,31453,IAH,"Houston, TX",TX,48,Texas,74
2,11973,1197302,31973,GPT,"Gulfport/Biloxi, MS",MS,28,Mississippi,53
3,15370,1537002,34653,TUL,"Tulsa, OK",OK,40,Oklahoma,73
4,12915,1291503,31205,LCH,"Lake Charles, LA",LA,22,Louisiana,72


From a bussines perspective point of view I wouldn't impute null values, since, for example, DepTime might be null because the flight might have been cancelled and if we impute or drop that registry we might affect the future predictions or even the hypothesis

### Data Acquisition - Database Creation

```
# load the .env file variables
load_dotenv()

# 1) Connect to the database here using the SQLAlchemy's create_engine function
def conn_string():
    user = os.getenv('DB_USER')
    pwd = os.getenv('DB_PASSWORD')
    host = os.getenv('DB_HOST')
    db = os.getenv('DB_NAME')

    return f"postgresql://{user}:{pwd}@{host}/{db}"

def connect():
    global engine
    print("Starting connection...")
    engine = create_engine(conn_string()).execution_options(autocommit=True)
    engine.connect()

    return engine

connect()


########################################################################
# Define your dataset columns and their corresponding data types
columns = {
    'FlightDate': 'DATE',
    'Airline': 'VARCHAR(255)',
    'Origin': 'VARCHAR(255)',
    # Define other columns and their data types here
}

# Generate SQL statement to create table
create_table_sql = f"""
CREATE TABLE flights (
    {", ".join([f"{col} {dtype}" for col, dtype in columns.items()])}
);
"""

# Execute SQL statement to create table
engine.execute(create_table_sql)

# Now, let's say you have your data stored in a pandas DataFrame called 'df'
# You can iterate over the DataFrame rows and generate SQL statements to insert data
for index, row in df.iterrows():
    insert_sql = f"""
    INSERT INTO flights ({", ".join(columns.keys())}) VALUES 
    ({", ".join([str(row[col]) for col in columns.keys()])});
    """
    # Execute SQL statement to insert data
    engine.execute(insert_sql)

########################################################################

# 4) Use pandas to print one of the tables as dataframes using read_sql function
df = pd.read_sql('SELECT * FROM authors', engine)
print(df)
```

In [64]:
load_dotenv()

# 1) Connect to the database here using the SQLAlchemy's create_engine function
def conn_string():
    user = os.getenv('DB_USER')
    pwd = os.getenv('DB_PASSWORD')
    host = os.getenv('DB_HOST')
    db = os.getenv('DB_NAME')

    return f"postgresql://{user}:{pwd}@{host}/{db}"

def connect():
    global engine
    print("Starting connection...")
    engine = create_engine(conn_string()).execution_options(autocommit=True)
    engine.connect()

    return engine

connect()

Starting connection...


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
def map_to_bool(cols):
    mapping = {True: 1, False: 0}
    cols = list(cols)

    for c in cols:
        data[c] = data[c].map(mapping)

array([-1.700e+01, -1.000e+00, -3.000e+00, ...,  1.222e+03,  1.342e+03,
        1.306e+03])

In [ ]:
map_to_bool(['Cancelled', 'Diverted'])

In [ ]:
orgs_cols = {
    'OriginAirportID': 'INTEGER',
    'OriginAirportSeqID': 'INTEGER',
    'OriginCityMarketID': 'INTEGER',
    'Origin': 'VARCHAR(10)',
    'OriginCityName': 'VARCHAR(60)',
    'OriginState': 'VARCHAR(10)',
    'OriginStateFips': 'INTEGER',
    'OriginStateName': 'VARCHAR(60)',
    'OriginWac': 'INTEGER',
}
dests_cols = {
    'DestAirportID': 'INTEGER',
    'DestAirportSeqID': 'INTEGER',
    'DestCityMarketID': 'INTEGER',
    'Dest': 'VARCHAR(10)',
    'DestCityName': 'VARCHAR(60)',
    'DestState': 'VARCHAR(10)',
    'DestStateFips': 'INTEGER',
    'DestStateName': 'VARCHAR(60)',
    'DestWac': 'INTEGER',
}

main_cols = {
    'FLIGHT_ID': 'INT PRIMARY KEY',
    'FlightDate': 'DATE',
    'Airline': 'VARCHAR(100)',
    'Cancelled': 'BOOLEAN NOT NULL',
    'Diverted': 'BOOLEAN NOT NULL',
    'DepTime': 'TIME',
    'DepDelayMinutes': 'NUMERIC',
    'DepDelay': 'NUMERIC',
    'ArrTime': 'TIME',
    'ArrDelayMinutes': 'NUMERIC',
    'AirTime': 'NUMERIC',
    'Distance': 'NUMERIC',
    'Year': 'INT',
    'Quarter': 'INT',
    'Month': 'INT',
    'DayofMonth': 'INT',
    'DayOfWeek': 'INT',
    'Marketing_Airline_Network': 'VARCHAR(10)',
    'Operated_or_Branded_Code_Share_Partners': 'VARCHAR(20)',
    'DOT_ID_Marketing_Airline': 'INT',
    'IATA_Code_Marketing_Airline': 'VARCHAR(10)',
    'Flight_Number_Marketing_Airline': 'INT',
    'Operating_Airline': 'VARCHAR(5)',
    'DOT_ID_Operating_Airline': 'INT',
    'IATA_Code_Operating_Airline': 'VARCHAR(5)',
    'Tail_Number': 'VARCHAR(20)', 
    'Flight_Number_Operating_Airline': 'INT',
    'OriginAirportID': 'INT',
    'DestAirportID': 'INT',
    'WheelsOff': 'TIME',
    'WheelsOn': 'TIME',
    'ArrDelay': 'NUMERIC',
    'DivAirportLandings': 'INT' 
}

In [ ]:
# Generate SQL statement to create table
create_main_table = f"""
CREATE TABLE flights (
    {", ".join([f"{col} {dtype}" for col, dtype in main_cols.items()])}
);
"""
create_origins_table = f"""
CREATE TABLE flights (
    {", ".join([f"{col} {dtype}" for col, dtype in orgs_cols.items()])}
);
"""
create_dest_table = f"""
CREATE TABLE flights (
    {", ".join([f"{col} {dtype}" for col, dtype in dests_cols.items()])}
);
"""

# Execute SQL statement to create table
engine.execute(create_main_table)
engine.execute(create_origins_table)
engine.execute(create_dest_table)

# Now, let's say you have your data stored in a pandas DataFrame called 'df'
# You can iterate over the DataFrame rows and generate SQL statements to insert data
for index, row in data.iterrows():
    insert_main_sql = f"""
    INSERT INTO flights ({", ".join(main_cols.keys())}) VALUES 
    ({", ".join([str(row[col]) for col in main_cols.keys()])});
    """
    # Execute SQL statement to insert data
    engine.execute(insert_main_sql)

for index, row in origins.iterrows():
    insert_origins_sql = f"""
    INSERT INTO flights ({", ".join(orgs_cols.keys())}) VALUES 
    ({", ".join([str(row[col]) for col in orgs_cols.keys()])});
    """
    # Execute SQL statement to insert data
    engine.execute(insert_main_sql)

for index, row in destinations.iterrows():
    insert_dests_sql = f"""
    INSERT INTO flights ({", ".join(dests_cols.keys())}) VALUES 
    ({", ".join([str(row[col]) for col in dests_cols.keys()])});
    """
    # Execute SQL statement to insert data
    engine.execute(insert_main_sql)


### Data Acquisition - Queries to obtain the data